# Project 3

- **Dataset(s) to be used:** [SIPA alumni employment data](https://www.sipa.columbia.edu/pathways-careers/employment-statistics)
- **Analysis question:** Is international students facing difficulty in finding jobs in U.S.?
- **Columns that will (likely) be used:**
  - International
  - Outcome
  - Job Country
  - Citizenship Status
  - OSA Country of Citizenship Lookup
- **Hypothesis:**
  - international student is less easy to be employed in U.S.
    - Citizenship status is one of the reasons
    - Primary language is one of the reasons

In [1]:
# import library
import pandas as pd

df = pd.read_excel(
    "D:/Python/PythonProject/Project 3/2023 Outcome Survey Data_Aidan.xlsx"
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 97 columns):
 #   Column                                                                                    Non-Null Count  Dtype  
---  ------                                                                                    --------------  -----  
 0   International                                                                             638 non-null    object 
 1   Filled out survey                                                                         638 non-null    object 
 2   APSIA Job Category                                                                        575 non-null    object 
 3   APSIA Sector Category                                                                     372 non-null    object 
 4   Date Reported - NEW                                                                       216 non-null    object 
 5   Outcome                                                  

According to the output, the non-null count is better than what I expected. They are mostly concentrated in division and salary related column. To develop a valuble hypothesis and find a feasible realizing route, I need to circumvent these columns. 

In [2]:
# data clean
df_reported = df[df["Outcome"] != "Unreported"]

df_counts = (
    df_reported.groupby(["International", "Outcome"]).size().reset_index(name="count")
)

# calculate proportion
df_counts["pct"] = df_counts.groupby("International")["count"].transform(
    lambda x: x / x.sum()
)

In [3]:
# plot
import plotly.express as px
from IPython.display import HTML

fig = px.bar(
    df_counts,
    x="Outcome",
    y="pct",
    color="International",
    barmode="group",
    text="pct",
    title="Outcome Distribution Within International vs Non-International Groups",
)

fig.update_layout(yaxis_title="Proportion within Group", xaxis_title="Outcome")

fig.update_traces(texttemplate="%{text:.1%}")
HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


Luckily, only few students were still seeking employment. In the first quick look of the employment data, there is no significant difference between two groups, most of the students can find a job after studying in SIPA. 

Now let's turn to the job location, as though international students can find a job in their own country, they may still have difficulty to find a job in U.S.

In [4]:
df_clean = df_reported.copy()

# delete Pursuing Further Education
df_clean = df_clean[df_clean["Outcome"] == "Job"]

df_clean = df_clean[df_clean["Job Country"].notna()]

# Classify Job Country：US vs Non-US
df_clean["JobCountry_US"] = df_clean["Job Country"].apply(
    lambda x: "United States" if x == "United States" else "Non-US"
)

df_counts = (
    df_clean.groupby(["International", "JobCountry_US"])
    .size()
    .reset_index(name="count")
)

df_counts["pct"] = df_counts.groupby("International")["count"].transform(
    lambda x: x / x.sum()
)

In [5]:
fig = px.bar(
    df_counts,
    x="International",
    y="pct",
    color="JobCountry_US",
    barmode="relative",
    text="pct",
    title="Job Country Distribution (US vs Non-US) by International Status",
)

fig.update_traces(texttemplate="%{text:.1%}", textposition="inside")

fig.update_layout(yaxis_title="Proportion", yaxis=dict(range=[0, 1]))

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))

The hypothesis is confirmed. Apparently there is a difference between US and Non-Us students, so what cause this difference? First I inspect the citizenship status, as US company are less willing to hire non-US citizens as they do not want to afford the high H1B fee.

In [6]:
# data clean
df_clean_status = df_clean[df_clean["Citizenship Status"].notna()]

In [7]:
# count
df_counts_status = (
    df_clean_status.groupby(["Citizenship Status", "JobCountry_US"])
    .size()
    .reset_index(name="count")
)

df_counts_status["pct"] = df_counts_status.groupby("Citizenship Status")[
    "count"
].transform(lambda x: x / x.sum())


In [8]:
fig = px.bar(
    df_counts_status,
    x="Citizenship Status",
    y="pct",
    color="JobCountry_US",
    barmode="group",
    text="pct",
    title="Job Country (US vs Non-US) Distribution by Citizenship Status",
)

fig.update_layout(
    xaxis_title="Citizenship Status", yaxis_title="Proportion within Group"
)

fig.update_traces(texttemplate="%{text:.1%}")
HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


The data clearly indicates that international students (F-1, J-1, H-1) are significantly less likely to obtain U.S.-based jobs compared to U.S. citizens or permanent residents, suggesting they face considerable difficulty securing employment in the United States.

Next, the reason why international students do not stay in US could also be the lack of English speaking ability. So I divide the country into English-speaking and Non-English speaking to find if there is any difference.

To define which country is English speaking, I use ChatGPT to help me build a list of English speaking countries according to the [List of countries and territories where English is an official language](https://en.wikipedia.org/wiki/List_of_countries_and_territories_where_English_is_an_official_language?utm_source=chatgpt.com) on wikipedia.

In [9]:
df_clean_language = df_clean.copy()

# data clean
df_clean_language["citizenship_country_clean"] = (
    df_clean_language["OSA Country of Citizenship Lookup"]
    .astype(str)
    .str.strip()
    .str.upper()
)

# repeated country name
alias_mapping = {
    "UNITED STATES (USA)": "UNITED STATES",
    "UNITED STATES OF AMERICA": "UNITED STATES",
}
df_clean_language["citizenship_country_clean"] = df_clean_language[
    "citizenship_country_clean"
].replace(alias_mapping)

# English country list
english_countries = {
    "ANTIGUA AND BARBUDA",
    "AUSTRALIA",
    "BAHAMAS",
    "BARBADOS",
    "BELIZE",
    "BOTSWANA",
    "CAMEROON",
    "CANADA",
    "DOMINICA",
    "ESWATINI",
    "FIJI",
    "GAMBIA",
    "GHANA",
    "GRENADA",
    "GUYANA",
    "INDIA",
    "IRELAND",
    "JAMAICA",
    "KENYA",
    "KIRIBATI",
    "LESOTHO",
    "LIBERIA",
    "MALAWI",
    "MALTA",
    "MARSHALL ISLANDS",
    "MAURITIUS",
    "MICRONESIA",
    "NAMIBIA",
    "NAURU",
    "NEW ZEALAND",
    "NIGERIA",
    "PAKISTAN",
    "PALAU",
    "PAPUA NEW GUINEA",
    "PHILIPPINES",
    "RWANDA",
    "SAINT KITTS AND NEVIS",
    "SAINT LUCIA",
    "SAINT VINCENT AND THE GRENADINES",
    "SAMOA",
    "SEYCHELLES",
    "SIERRA LEONE",
    "SINGAPORE",
    "SOLOMON ISLANDS",
    "SOUTH AFRICA",
    "SOUTH SUDAN",
    "SRI LANKA",
    "SUDAN",
    "TANZANIA",
    "TONGA",
    "TRINIDAD AND TOBAGO",
    "TUVALU",
    "UGANDA",
    "UNITED KINGDOM",
    "UNITED STATES",
    "VANUATU",
    "ZAMBIA",
    "ZIMBABWE",
}

# classify
df_clean_language["citizenship_english_country"] = df_clean_language[
    "citizenship_country_clean"
].apply(lambda x: "English-speaking" if x in english_countries else "Non-English")

# count proportion
df_counts_language = (
    df_clean_language.groupby(["citizenship_english_country", "JobCountry_US"])
    .size()
    .reset_index(name="count")
)

df_counts_language["pct"] = df_counts_language.groupby("citizenship_english_country")[
    "count"
].transform(lambda x: x / x.sum())


In [11]:
fig = px.bar(
    df_counts_language,
    x="citizenship_english_country",
    y="pct",
    color="JobCountry_US",
    barmode="relative",
    text="pct",
    title="Job Country (US vs Non-US) by Citizenship Language Background",
)

fig.update_traces(texttemplate="%{text:.1%}", textposition="inside")

fig.update_layout(
    yaxis_title="Proportion",
    xaxis_title="Citizenship Country Language Background",
    yaxis=dict(range=[0, 1]),
    bargap=0.2,
)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))


From the chart above, it is possible that Language background is one of the factors that make the gap.

Overall, an international student may find difficulty in applying jobs in U.S. because of their citizenship status and language.